# 01 — Setup & Ingest (PDF → Chroma Index)
This notebook parses PDFs (page-level), embeds chunks, and builds a Chroma index.

In [2]:
#%pip install pypdf
%pip install -U -q pip wheel setuptools "chromadb>=0.5.5"
#%pip install -p 

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os, glob, uuid
from dotenv import load_dotenv
from pypdf import PdfReader
import chromadb
import numpy as np


